In [1]:
# -*- coding: utf-8 -*-
import os
import warnings
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
from geopy.distance import geodesic
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import statsmodels.api as sm

warnings.simplefilter("ignore", ConvergenceWarning)

# =============================
# Configurações
# =============================
STATIONS_CSV = "../data/all_stations.csv"
UNI_CSV = "./universidades_federais_coords_26_estados.csv"
CLEANED_BASE = "../data/cleaned_data"
YEARS = [2019, 2020, 2021, 2022, 2023, 2024]
RADIUS_KM = 35.0

FORECAST_PARAMETER = "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"
EXOG_VARS = [
    "RADIACAO GLOBAL (KJ/m²)",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",
    "UMIDADE RELATIVA DO AR, HORARIA (%)",
]

RESAMPLE_RULE = "D"     # média diária
SPLIT_TRAIN = 0.80      # 80/20 treino/teste
ARIMA_ORDER = (3, 0, 1) # ARIMA(3,0,1)
FOURIER_PERIOD = 365    # dias
FOURIER_K = 4           # nº de harmônicos (gera 2*K colunas)

# =============================
# Utilidades básicas
# =============================
def calculate_distance_from_point_to_station(row, given_point_coord):
    station_coord = (row["LATITUDE:"], row["LONGITUDE:"])
    return geodesic(station_coord, given_point_coord).kilometers

def smape(y_true, y_pred):
    y_true, y_pred = pd.Series(y_true), pd.Series(y_pred)
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

def create_fourier_terms(t, period, num_terms):
    terms = []
    for i in range(1, num_terms + 1):
        terms.append(np.sin(2 * np.pi * i * t / period))
        terms.append(np.cos(2 * np.pi * i * t / period))
    return np.column_stack(terms)

# =============================
# Leitura de estações e dados
# =============================
def load_all_stations() -> pd.DataFrame:
    df_all_stations = pd.read_csv(STATIONS_CSV, decimal=",", sep=";")
    df_all_stations.columns = [c.strip() for c in df_all_stations.columns]
    return df_all_stations

def load_weather_for_station_filename(filename_2019: str) -> Optional[pd.DataFrame]:
    """
    Usa o nome-base de 2019 e monta os caminhos 2019–2024.
    Concatena o que existir, e corrige 'RADIACAO GLOBAL (Kj/m²)' -> 'RADIACAO GLOBAL (KJ/m²)'.
    """
    dfs = []
    for year in YEARS:
        year_dir = f"{year}_cleaned"
        target = filename_2019.replace("2019", str(year))
        path = os.path.join(CLEANED_BASE, year_dir, target)
        if os.path.exists(path):
            try:
                df_weather_data = pd.read_csv(path, decimal=".", sep=";")
                # 🔧 correção de nomenclatura (Kj → KJ)
                if 'RADIACAO GLOBAL (Kj/m²)' in df_weather_data.columns:
                    df_weather_data.rename(
                        columns={'RADIACAO GLOBAL (Kj/m²)': 'RADIACAO GLOBAL (KJ/m²)'},
                        inplace=True
                    )
                dfs.append(df_weather_data)
            except Exception as e:
                print(f"[WARN] Falha ao ler {path}: {e}")
    if not dfs:
        return None
    return pd.concat(dfs, ignore_index=True)

def build_daily_panel_from_stations(df_nearest_stations: pd.DataFrame) -> Optional[pd.DataFrame]:
    """
    Retorna um DataFrame diário (index=Data) com média das variáveis numéricas
    das estações do raio. Mantém target + exógenas, faz ffill().
    """
    collected = []
    for filename in df_nearest_stations["Arquivo"]:
        dfw = load_weather_for_station_filename(filename)
        if dfw is not None:
            collected.append(dfw)
    if not collected:
        return None

    df = pd.concat(collected, ignore_index=True)
    if "Hora UTC" in df.columns:
        df = df.drop(columns=["Hora UTC"])
    df["Data"] = pd.to_datetime(df["Data"], errors="coerce")
    df = df.dropna(subset=["Data"]).sort_values("Data").set_index("Data")

    daily = df.resample(RESAMPLE_RULE).mean(numeric_only=True)

    # garantir colunas requeridas
    needed = [FORECAST_PARAMETER] + EXOG_VARS
    missing = [c for c in needed if c not in daily.columns]
    if missing:
        raise KeyError(f"Colunas faltando após resample: {missing}\nDisponíveis: {list(daily.columns)[:12]} ...")

    daily = daily.ffill()
    return daily

def series_for_coord(coord: Tuple[float, float], df_all_stations: pd.DataFrame) -> Optional[pd.DataFrame]:
    df = df_all_stations.copy()
    df["Distancia"] = df.apply(lambda r: calculate_distance_from_point_to_station(r, coord), axis=1)
    df_nearest = df[df["Distancia"] < RADIUS_KM]
    if df_nearest.empty:
        df_nearest = df.sort_values("Distancia").head(1)  # fallback: estação mais próxima
    return build_daily_panel_from_stations(df_nearest)

# =============================
# Exógenas com máscaras (sem vazamento)
# =============================
def _profiles_from_train(df_training: pd.DataFrame, exog_vars):
    prof_mmdd = (
        df_training.assign(__mm__=df_training.index.month, __dd__=df_training.index.day)
                   .groupby(["__mm__", "__dd__"])[exog_vars]
                   .median()
    )
    prof_doy = (
        df_training.assign(__doy__=df_training.index.dayofyear)
                   .groupby("__doy__")[exog_vars]
                   .median()
    )
    return prof_mmdd, prof_doy

def _impute_with_profiles(s: pd.Series, mmdd_index, doy_index, prof_mmdd, prof_doy, train_col_median):
    need = s.isna()
    if need.any():
        s.loc[need] = prof_mmdd[s.name].reindex(mmdd_index[need]).values
    need = s.isna()
    if need.any():
        s.loc[need] = prof_doy[s.name].reindex(doy_index[need]).values
    need = s.isna()
    if need.any():
        s.loc[need] = train_col_median
    return s

def prepare_exog_with_masks(df_training: pd.DataFrame,
                            df_test: pd.DataFrame,
                            exog_vars,
                            fourier_period: int = FOURIER_PERIOD,
                            fourier_k: int = FOURIER_K):
    """
    Retorna X_train, X_test, feature_names com:
      [exógenas imputadas | máscaras *_MISSING | Fourier]
    Máscaras: 1 quando o valor ORIGINAL era NaN, 0 caso contrário.
    Imputação do TESTE usa apenas estatísticas do TREINO (sem vazamento).
    """
    # máscaras originais
    train_masks = {f"{col}_MISSING": df_training[col].isna().astype(int) for col in exog_vars}
    test_masks  = {f"{col}_MISSING": df_test[col].isna().astype(int)      for col in exog_vars}

    prof_mmdd, prof_doy = _profiles_from_train(df_training, exog_vars)

    # índices auxiliares
    mmdd_tr = pd.MultiIndex.from_arrays([df_training.index.month, df_training.index.day], names=["__mm__", "__dd__"])
    mmdd_te = pd.MultiIndex.from_arrays([df_test.index.month,     df_test.index.day    ], names=["__mm__", "__dd__"])
    doy_tr  = ((df_training.index.dayofyear - 1) % 365) + 1
    doy_te  = ((df_test.index.dayofyear     - 1) % 365) + 1

    tr_imp = df_training[exog_vars].copy()
    te_imp = df_test[exog_vars].copy()

    # imputação treino
    for col in exog_vars:
        tr_imp[col] = _impute_with_profiles(
            tr_imp[col], mmdd_tr, doy_tr, prof_mmdd, prof_doy,
            train_col_median=df_training[col].median(skipna=True)
        )
    # imputação teste (sem vazamento)
    for col in exog_vars:
        te_imp[col] = _impute_with_profiles(
            te_imp[col], mmdd_te, doy_te, prof_mmdd, prof_doy,
            train_col_median=df_training[col].median(skipna=True)
        )

    # Fourier
    n_tr, n_te = len(df_training), len(df_test)
    t_tr = np.arange(n_tr)
    t_te = np.arange(n_tr, n_tr + n_te)
    F_tr = create_fourier_terms(t_tr, fourier_period, fourier_k)
    F_te = create_fourier_terms(t_te, fourier_period, fourier_k)
    fourier_cols = [f"F_{k}_{fn}" for k in range(1, fourier_k + 1) for fn in ("sin", "cos")]

    # monta X
    X_train = np.hstack([
        tr_imp.to_numpy(),
        np.column_stack([train_masks[m].to_numpy() for m in train_masks]),
        F_tr
    ])
    X_test  = np.hstack([
        te_imp.to_numpy(),
        np.column_stack([test_masks[m].to_numpy() for m in test_masks]),
        F_te
    ])

    feature_names = list(exog_vars) + list(train_masks.keys()) + fourier_cols
    return X_train, X_test, feature_names

# =============================
# ARIMA(3,0,1) com exógenas + Fourier + calibração
# =============================
def debias_with_calibration(model_fit, df_training, forecast_parameter, X_train_final, forecast_vals, calib_days=60):
    start = max(0, len(df_training) - calib_days)
    pred_cal = model_fit.get_prediction(
        start=df_training.index[start],
        end=df_training.index[-1],
        exog=X_train_final[start:]
    ).predicted_mean
    y_cal = df_training[forecast_parameter].iloc[start:]
    ME = (y_cal - pred_cal).mean()
    forecast_bias_fixed = forecast_vals + ME
    X = sm.add_constant(pred_cal.values)
    a, b = sm.OLS(y_cal.values, X).fit().params
    forecast_linear_cal = a + b * forecast_vals
    return {"ME": ME, "forecast_bias_fixed": forecast_bias_fixed, "a": a, "b": b, "forecast_linear_cal": forecast_linear_cal}

def arima_forecast_with_fourier_terms_exog(df_training: pd.DataFrame,
                                           df_test: pd.DataFrame,
                                           forecast_parameter: str,
                                           trend: str = "ct",
                                           calib_days: int = 60) -> pd.Series:
    X_train, X_test, _ = prepare_exog_with_masks(df_training, df_test, EXOG_VARS, FOURIER_PERIOD, FOURIER_K)

    model = ARIMA(df_training[forecast_parameter], exog=X_train, order=ARIMA_ORDER, trend=trend)
    model_fit = model.fit()

    forecast_vals = model_fit.forecast(steps=len(df_test), exog=X_test)
    cal = debias_with_calibration(model_fit, df_training, forecast_parameter, X_train, forecast_vals, calib_days=calib_days)

    forecast_corrected = pd.Series(cal["forecast_bias_fixed"], index=df_test.index, name="forecast")
    return forecast_corrected

# =============================
# Pipeline principal
# =============================
def run_for_coords(coords: List[Tuple[float, float]]) -> Dict[Tuple[float, float], Dict]:
    df_all = load_all_stations()
    results: Dict[Tuple[float, float], Dict] = {}

    for coord in coords:
        daily = series_for_coord(coord, df_all)
        if daily is None:
            results[coord] = {"ok": False, "reason": "Sem dados para essa coordenada."}
            continue

        split_idx = int(len(daily) * SPLIT_TRAIN)
        df_training = daily.iloc[:split_idx].copy()
        df_test     = daily.iloc[split_idx:].copy()

        # sanity check
        needed = [FORECAST_PARAMETER] + EXOG_VARS
        if any(col not in df_training.columns for col in needed) or any(col not in df_test.columns for col in needed):
            results[coord] = {"ok": False, "reason": "Colunas necessárias ausentes (target/exógenas)."}
            continue

        forecast = arima_forecast_with_fourier_terms_exog(df_training, df_test, FORECAST_PARAMETER)

        # métricas (RMSE com debias; MSE; sMAPE)
        mse = mean_squared_error(df_test[FORECAST_PARAMETER], forecast)
        rmse = mse ** 0.5
        smape_val = smape(df_test[FORECAST_PARAMETER], forecast)

        results[coord] = {
            "ok": True,
            "periodo": f"{daily.index.min().date()} → {daily.index.max().date()}",
            "n_total": len(daily),
            "n_train": len(df_training),
            "n_test": len(df_test),
            "RMSE": float(rmse),           # já com debias
            "MSE": float(mse),
            "sMAPE": float(smape_val),
            "forecast": forecast,          # série prevista (apenas janela de teste)
        }

    return results

# =============================
# Rodar para TODAS as universidades e imprimir tabela
# =============================
# Lê CSV das universidades
try:
    df_uni = pd.read_csv(UNI_CSV)  # tenta separador padrão
except Exception:
    df_uni = pd.read_csv(UNI_CSV, sep=";", decimal=",")  # fallback

df_uni.columns = [c.strip() for c in df_uni.columns]
assert {"Universidade", "Estado", "Latitude", "Longitude"}.issubset(df_uni.columns), \
    f"CSV de universidades não tem as colunas esperadas. Tenho: {df_uni.columns.tolist()}"

# Monta coords na mesma ordem do CSV
coords = [(float(lat), float(lon)) for lat, lon in zip(df_uni["Latitude"], df_uni["Longitude"])]

# Executa
results = run_for_coords(coords)

# Tabela final
rows = []
for i, coord in enumerate(coords):
    uni = df_uni.iloc[i]["Universidade"]
    uf  = df_uni.iloc[i]["Estado"]
    r   = results.get(coord, {})
    if r.get("ok"):
        rows.append({
            "Universidade": uni,
            "Estado": uf,
            "Latitude": round(coord[0], 6),
            "Longitude": round(coord[1], 6),
            "Período": r["periodo"],
            "Obs Totais": r["n_total"],
            "Treino": r["n_train"],
            "Teste": r["n_test"],
            "RMSE (debias)": round(r["RMSE"], 3),
            "MSE": round(r["MSE"], 3),
            "sMAPE (%)": round(r["sMAPE"], 3),
        })
    else:
        rows.append({
            "Universidade": uni,
            "Estado": uf,
            "Latitude": round(coord[0], 6),
            "Longitude": round(coord[1], 6),
            "Status": "Falhou",
            "Motivo": r.get("reason", "Sem dados após filtro/transformações"),
        })

df_resultados = pd.DataFrame(rows)
sort_cols = [c for c in ["Status", "Estado", "Universidade"] if c in df_resultados.columns]
df_resultados = df_resultados.sort_values(by=sort_cols).reset_index(drop=True)

print("\n=== RESULTADOS POR UNIVERSIDADE (ARIMA(3,0,1) + EXÓGENAS + FOURIER + MASKS) ===")
print(df_resultados)

# opcional: salvar
# df_resultados.to_csv("resultados_arima_universidades.csv", index=False, sep=";")



=== RESULTADOS POR UNIVERSIDADE (ARIMA(3,0,1) + EXÓGENAS + FOURIER + MASKS) ===
                                         Universidade Estado  Latitude  \
0   Universidade Federal do Acre (UFAC) – Campus R...     AC  -9.95784   
1   Universidade Federal de Alagoas (UFAL) – Campu...     AL  -9.55232   
2   Universidade Federal do Amazonas (UFAM) – Camp...     AM  -3.09303   
3   Universidade Federal do Amapá (UNIFAP) – Campu...     AP  -0.00624   
4   Universidade Federal da Bahia (UFBA) – Campus ...     BA -13.00178   
5   Universidade Federal do Ceará (UFC) – Campus d...     CE  -3.74455   
6   Universidade Federal do Espírito Santo (UFES) ...     ES -20.27706   
7   Universidade Federal de Goiás (UFG) – Campus S...     GO -16.60387   
8   Universidade Federal do Maranhão (UFMA) – Cida...     MA  -2.55932   
9   Universidade Federal de Minas Gerais (UFMG) – ...     MG -19.87069   
10  Universidade Federal de Mato Grosso do Sul (UF...     MS -20.50170   
11  Universidade Federal de Mat

In [ ]:
# =======================================================
# TABELA COMPACTA PARA OVERLEAF (resize + colunas reduzidas)
# =======================================================
def print_latex_table_compact(df: pd.DataFrame,
                              caption: str = "Resultados por universidade (compacto)",
                              label: str = "tab:resultados_compacto",
                              float_format: str = "%.3f",
                              longtable: bool = False):
    """
    Imprime no console uma tabela LaTeX compacta com:
      Sigla | Campus | Estado | RMSE (debias) | sMAPE (%)
    A tabela é envolvida por \resizebox{\textwidth}{!}{...} para caber na página.
    Não altera df_resultados original e não salva arquivos.
    """

    import re
    df2 = df.copy()

    # ---------- extratores de Sigla e Campus a partir da coluna "Universidade" ----------
    def _extrair_sigla(nome_uni: str) -> str:
        # tenta pegar uma sigla já em maiúsculas (ex: UFMG, UFPR, IFRN...)
        match = re.findall(r'\b[A-Z]{2,}\b', nome_uni or "")
        if match:
            return match[0]
        # fallback: iniciais de palavras significativas
        palavras = [p for p in re.split(r"\s+", nome_uni or "") if p and p[0].isalpha()]
        iniciais = "".join([p[0].upper() for p in palavras if len(p) > 2])
        return iniciais[:6] or "—"

    def _extrair_campus(nome_uni: str) -> str:
        if not nome_uni:
            return "Sede"
        # padrões comuns: " - Campus X", " – Campus X", "... (Campus X)"
        if re.search(r'Campus', nome_uni, flags=re.IGNORECASE):
            # pega tudo após a palavra Campus
            pos = re.split(r'(?i)Campus', nome_uni, maxsplit=1)[-1]
            pos = re.sub(r'^[\s:\-–]+', '', pos).strip()
            # remove parênteses finais, se vier "Campus X)"; mantém só o nome
            pos = re.sub(r'^\(?\s*(.*?)\s*\)?$', r'\1', pos)
            return pos if pos else "Sede"
        # se não houver "Campus", assume sede
        return "Sede"

    if "Universidade" in df2.columns:
        df2["Sigla"] = df2["Universidade"].astype(str).apply(_extrair_sigla)
        df2["Campus"] = df2["Universidade"].astype(str).apply(_extrair_campus)
    else:
        # se não existir a coluna (caso raro), ainda assim cria vazias
        df2["Sigla"] = ""
        df2["Campus"] = "Sede"

    # ---------- seleção de colunas compactas ----------
    cols_final = ["Sigla", "Campus", "Estado", "RMSE (debias)", "sMAPE (%)"]
    cols_final = [c for c in cols_final if c in df2.columns]
    df2 = df2[cols_final].copy()

    # renomeia "sMAPE (%)" -> "sMAPE (\%)" para LaTeX
    df2.rename(columns={c: c.replace("%", r"\%") for c in df2.columns}, inplace=True)

    # ---------- formatação numérica consistente ----------
    def _fmt(x):
        try:
            return float_format % float(x)
        except Exception:
            return x
    df2 = df2.applymap(_fmt)

    # ---------- monta LaTeX com pandas ----------
    tbl = df2.to_latex(index=False, escape=True, longtable=longtable, bold_rows=False)

    # ---------- imprime com resizebox para caber na página ----------
    # (precisa de \usepackage{graphicx} no preâmbulo)
    if longtable:
        # se preferir longtable, não há "table float"; pode combinar com landscape se quiser
        # ainda assim encapsulamos o conteúdo do longtable em resizebox é incomum,
        # então aqui imprimimos o longtable puro e confiamos na quebra automática.
        print("\n=== BLOCO LaTeX (COMPACTO) ===\n")
        print(
            f"\\begin{longtable}{{{tbl.splitlines()[0].split('{')[-1].split('}')[0]}}}\n"
            f"\\caption{{{caption}}}\\\\\n"
            f"\\label{{{label}}}\n" +
            "\n".join(tbl.splitlines()[1:])  # resto do conteúdo gerado
        )
    else:
        print("\n=== BLOCO LaTeX (COMPACTO) ===\n")
        print(
            "\\begin{table}[!ht]\n"
            "\\centering\n"
            "\\resizebox{\\textwidth}{!}{%\n"
            f"{tbl}\n"
            "}%\n"
            f"\\caption{{{caption}}}\n"
            f"\\label{{{label}}}\n"
            "\\end{table}\n"
        )

# === chamada direta (sem salvar nada) ===
print_latex_table_compact(
    df_resultados,
    caption="Resultados por universidade (Sigla, Campus, Estado, RMSE, sMAPE)",
    label="tab:resultados_compacto",
    longtable=False,  # mude para True se preferir quebra automática por páginas
)


In [ ]:
# =======================================================
# MÉDIAS GERAIS DAS MÉTRICAS
# =======================================================
def print_metric_means(df: pd.DataFrame):
    """Calcula e exibe as médias de RMSE, MSE e sMAPE no console."""
    metricas = ["RMSE (debias)", "MSE", "sMAPE (%)"]
    disponiveis = [m for m in metricas if m in df.columns]

    medias = df[disponiveis].mean(numeric_only=True)
    print("\n=== MÉDIAS GERAIS DAS MÉTRICAS ===")
    for metrica, valor in medias.items():
        print(f"{metrica}: {valor:.3f}")

print_metric_means(df_resultados)


# =======================================================
# MÉDIAS DAS MÉTRICAS POR REGIÃO DO BRASIL
# =======================================================
def print_metric_means_by_region(df: pd.DataFrame):
    """Calcula e exibe as médias de RMSE, MSE e sMAPE por região brasileira."""

    # Mapeamento das UFs para regiões
    regioes = {
        "Norte": ["AC", "AP", "AM", "PA", "RO", "RR", "TO"],
        "Nordeste": ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
        "Centro-Oeste": ["DF", "GO", "MT", "MS"],
        "Sudeste": ["ES", "MG", "RJ", "SP"],
        "Sul": ["PR", "RS", "SC"]
    }

    metricas = ["RMSE (debias)", "MSE", "sMAPE (%)"]
    disponiveis = [m for m in metricas if m in df.columns]

    # adiciona coluna "Região"
    df = df.copy()
    df["Região"] = df["Estado"].map({
        uf: regiao for regiao, ufs in regioes.items() for uf in ufs
    })

    # calcula médias
    medias_regiao = df.groupby("Região")[disponiveis].mean(numeric_only=True).round(3)

    print("\n=== MÉDIAS DAS MÉTRICAS POR REGIÃO ===")
    print(medias_regiao)

print_metric_means_by_region(df_resultados)


In [2]:
# =======================================================
# REDE NEURAL (MLP) PARA TODAS AS UNIVERSIDADES (TEMPERATURA)
# =======================================================
import os
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# -----------------------------
# CONFIGURAÇÕES
# -----------------------------
STATIONS_CSV = "../data/all_stations.csv"
CLEANED_BASE = "../data/cleaned_data"
UNI_CSV = "./universidades_federais_coords_26_estados.csv"
YEARS = [2019, 2020, 2021, 2022, 2023, 2024]
RADIUS_KM = 35.0
RESAMPLE_RULE = "D"    # média diária
SPLIT_TRAIN = 0.80     # 80/20 treino/teste
FORECAST_PARAMETER = "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"

N_PAST = 7
EPOCHS = 100
BATCH_SIZE = 32
LR = 1e-3

torch.manual_seed(1)

# -----------------------------
# FUNÇÕES AUXILIARES
# -----------------------------
def smape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float).ravel()
    y_pred = np.asarray(y_pred, dtype=float).ravel()
    denom = (np.abs(y_pred) + np.abs(y_true))
    denom[denom == 0] = 1e-12
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / denom)

def safe_align_and_mask(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float).ravel()
    y_pred = np.asarray(y_pred, dtype=float).ravel()
    n = min(len(y_true), len(y_pred))
    y_true = y_true[:n]
    y_pred = y_pred[:n]
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    return y_true[mask], y_pred[mask]

def _calc_dist_from_coord(row, given_point_coord):
    station_coord = (row["LATITUDE:"], row["LONGITUDE:"])
    return geodesic(station_coord, given_point_coord).kilometers

def _load_all_stations():
    df = pd.read_csv(STATIONS_CSV, decimal=",", sep=";")
    df.columns = [c.strip() for c in df.columns]
    return df

def _load_weather_for_station_filename(filename_2019: str):
    dfs = []
    for year in YEARS:
        year_dir = f"{year}_cleaned"
        target = filename_2019.replace("2019", str(year))
        path = os.path.join(CLEANED_BASE, year_dir, target)
        if os.path.exists(path):
            try:
                dfw = pd.read_csv(path, decimal=".", sep=";")
                if 'RADIACAO GLOBAL (Kj/m²)' in dfw.columns:
                    dfw.rename(columns={'RADIACAO GLOBAL (Kj/m²)': 'RADIACAO GLOBAL (KJ/m²)'}, inplace=True)
                dfs.append(dfw)
            except Exception as e:
                print(f"[WARN] Falha ao ler {path}: {e}")
    if not dfs:
        return None
    return pd.concat(dfs, ignore_index=True)

def _build_daily_panel_from_stations(df_nearest_stations: pd.DataFrame):
    collected = []
    for filename in df_nearest_stations["Arquivo"]:
        dfw = _load_weather_for_station_filename(filename)
        if dfw is not None:
            collected.append(dfw)
    if not collected:
        return None

    df = pd.concat(collected, ignore_index=True)
    if "Hora UTC" in df.columns:
        df = df.drop(columns=["Hora UTC"])
    df["Data"] = pd.to_datetime(df["Data"], errors="coerce")
    df = df.dropna(subset=["Data"]).sort_values("Data").set_index("Data")
    daily = df.resample(RESAMPLE_RULE).mean(numeric_only=True)
    daily = daily.ffill().bfill()
    return daily if FORECAST_PARAMETER in daily.columns else None

def _series_for_coord(coord, df_all_stations):
    df = df_all_stations.copy()
    df["Distancia"] = df.apply(lambda r: _calc_dist_from_coord(r, coord), axis=1)
    df_nearest = df[df["Distancia"] < RADIUS_KM]
    if df_nearest.empty:
        df_nearest = df.sort_values("Distancia").head(1)
    return _build_daily_panel_from_stations(df_nearest)

def _make_windows(series_vals: np.ndarray, n_past: int):
    X, y = [], []
    for i in range(len(series_vals) - n_past):
        X.append(series_vals[i:i+n_past])
        y.append(series_vals[i+n_past])
    return np.array(X, dtype=float), np.array(y, dtype=float)

# -----------------------------
# REDE NEURAL CLIMÁTICA
# -----------------------------
class RedeClimatica(nn.Module):
    def __init__(self, n_past):
        super().__init__()
        self.fc1 = nn.Linear(n_past, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

def fit_predict_mlp(train_windows, train_targets, test_windows):
    X_train_t = torch.tensor(train_windows, dtype=torch.float32)
    y_train_t = torch.tensor(train_targets, dtype=torch.float32).unsqueeze(1)
    X_test_t  = torch.tensor(test_windows, dtype=torch.float32)

    train_ds = TensorDataset(X_train_t, y_train_t)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=False)

    model = RedeClimatica(train_windows.shape[1])
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)

    model.train()
    for _ in range(EPOCHS):
        for Xb, yb in train_loader:
            optimizer.zero_grad()
            out = model(Xb)
            loss = criterion(out, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_t).squeeze().numpy()
    return y_pred

# -----------------------------
# EXECUÇÃO PRINCIPAL
# -----------------------------
df_uni = pd.read_csv(UNI_CSV)
df_uni.columns = [c.strip() for c in df_uni.columns]
df_all_st = _load_all_stations()

print("\n=== RESULTADOS POR UNIVERSIDADE — REDE NEURAL (TEMPERATURA) ===")
rows_nn = []

for i, row in df_uni.iterrows():
    uni, uf = row["Universidade"], row["Estado"]
    coord = (row["Latitude"], row["Longitude"])

    daily = _series_for_coord(coord, df_all_st)
    if daily is None:
        print(f"[{uf}] {uni}: Falhou — sem dados.")
        continue

    y_daily = daily[FORECAST_PARAMETER].astype(float).ffill().bfill().dropna()
    if len(y_daily) < (365 // 2):
        print(f"[{uf}] {uni}: Série curta ({len(y_daily)} dias).")
        continue

    split_idx = int(len(y_daily) * SPLIT_TRAIN)
    scaler = MinMaxScaler()
    y_scaled = scaler.fit_transform(y_daily.to_numpy().reshape(-1, 1)).flatten()

    X_all, y_all = _make_windows(y_scaled, N_PAST)
    if len(y_all) < 10:
        print(f"[{uf}] {uni}: Poucos dados.")
        continue

    split_win = max(1, split_idx - N_PAST)
    X_train, y_train = X_all[:split_win], y_all[:split_win]
    X_test, y_test = X_all[split_win:], y_all[split_win:]

    y_pred = fit_predict_mlp(X_train, y_train, X_test)

    y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    y_true_clean, y_pred_clean = safe_align_and_mask(y_test_inv, y_pred_inv)
    if y_true_clean.size == 0:
        print(f"[{uf}] {uni}: sem pontos válidos.")
        continue

    mse = mean_squared_error(y_true_clean, y_pred_clean)
    rmse = float(mse ** 0.5)
    smape_val = float(smape(y_true_clean, y_pred_clean))

    print(f"[{uf}] {uni} | MSE={mse:.3f} | RMSE={rmse:.3f} °C | sMAPE={smape_val:.3f} %")

    rows_nn.append({
        "Universidade": uni,
        "Estado": uf,
        "MSE": round(float(mse), 3),
        "RMSE (debias)": round(rmse, 3),
        "sMAPE (%)": round(smape_val, 3)
    })

df_resultados_mlp = pd.DataFrame(rows_nn)
print("\n=== RESULTADOS FINAIS ===")
print(df_resultados_mlp)



=== RESULTADOS POR UNIVERSIDADE — REDE NEURAL (TEMPERATURA) ===
[AC] Universidade Federal do Acre (UFAC) – Campus Rio Branco | MSE=1.666 | RMSE=1.291 °C | sMAPE=3.759 %
[AL] Universidade Federal de Alagoas (UFAL) – Campus A.C. Simões (Maceió) | MSE=0.445 | RMSE=0.667 °C | sMAPE=1.966 %
[AP] Universidade Federal do Amapá (UNIFAP) – Campus Marco Zero (Macapá) | MSE=0.478 | RMSE=0.691 °C | sMAPE=1.805 %
[AM] Universidade Federal do Amazonas (UFAM) – Campus Manaus (Setor Sul) | MSE=1.459 | RMSE=1.208 °C | sMAPE=3.320 %
[BA] Universidade Federal da Bahia (UFBA) – Campus Ondina (Salvador) | MSE=0.436 | RMSE=0.660 °C | sMAPE=1.857 %
[CE] Universidade Federal do Ceará (UFC) – Campus do Pici (Fortaleza) | MSE=0.437 | RMSE=0.661 °C | sMAPE=1.284 %
[ES] Universidade Federal do Espírito Santo (UFES) – Campus Goiabeiras (Vitória) | MSE=1.489 | RMSE=1.220 °C | sMAPE=3.730 %
[GO] Universidade Federal de Goiás (UFG) – Campus Samambaia (Goiânia) | MSE=1.276 | RMSE=1.130 °C | sMAPE=3.371 %
[MA] Univers

In [8]:
# =======================================================
# MÉDIAS GERAIS DAS MÉTRICAS
# =======================================================
print_metric_means(df_resultados_mlp)


# =======================================================
# MÉDIAS DAS MÉTRICAS POR REGIÃO DO BRASIL
# =======================================================
print_metric_means_by_region(df_resultados_mlp)

print_latex_table_compact(
    df_resultados_mlp,
    caption="Resultados por universidade (Sigla, Campus, Estado, RMSE, sMAPE)",
    label="tab:resultados_compacto",
    longtable=False,  # mude para True se preferir quebra automática por páginas
)



=== MÉDIAS GERAIS DAS MÉTRICAS ===
RMSE (debias): 1.125
MSE: 1.592
sMAPE (%): 3.455

=== MÉDIAS DAS MÉTRICAS POR REGIÃO ===
              RMSE (debias)    MSE  sMAPE (%)
Região                                       
Centro-Oeste          1.753  3.267      4.835
Nordeste              0.551  0.351      1.413
Norte                 1.061  1.203      2.768
Sudeste               1.438  2.139      4.832
Sul                   1.948  3.818      7.970


/tmp/ipykernel_69084/868366476.py:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2 = df2.applymap(_fmt)



=== BLOCO LaTeX (COMPACTO) ===

\begin{table}[!ht]
\centering
\resizebox{\textwidth}{!}{%
\begin{tabular}{lllll}
\toprule
Sigla & Campus & Estado & RMSE (debias) & sMAPE (\textbackslash \%) \\
\midrule
UFAC & Rio Branco & AC & 1.291 & 3.759 \\
UFAL & A.C. Simões (Maceió & AL & 0.667 & 1.966 \\
UNIFAP & Marco Zero (Macapá & AP & 0.691 & 1.805 \\
UFAM & Manaus (Setor Sul & AM & 1.208 & 3.320 \\
UFBA & Ondina (Salvador & BA & 0.660 & 1.857 \\
UFC & do Pici (Fortaleza & CE & 0.661 & 1.284 \\
UFES & Goiabeiras (Vitória & ES & 1.220 & 3.730 \\
UFG & Samambaia (Goiânia & GO & 1.130 & 3.371 \\
UFMA & Sede & MA & 0.537 & 1.489 \\
UFMT & Cuiabá & MT & 2.051 & 4.993 \\
UFMS & Sede & MS & 2.078 & 6.141 \\
UFMG & Pampulha (Belo Horizonte & MG & 1.240 & 4.497 \\
UFPA & Guamá (Belém & PA & 0.741 & 1.977 \\
UFPB & I (João Pessoa & PB & 0.697 & 1.951 \\
UFPR & Sede & PR & 2.041 & 8.681 \\
UFPE & Sede & PE & 0.035 & 0.139 \\
UFPI & Ministro Petrônio Portella (Teresina & PI & 0.812 & 2.099 \\
UFRJ & Sed